In [1]:
#!/usr/bin/python -tt
# -*- coding: utf-8 -*-
"""
Created on Sat May 27 12:46:25 2017

@author: ehsanamid
"""

# -*- coding: utf-8 -*-
"""
Created on Fri May 26 12:17:14 2017

@author: ehsanamid
"""

import sys
from sklearn.neighbors import NearestNeighbors as knn
from sklearn.decomposition import TruncatedSVD
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
import pickle
from sklearn.externals import joblib

In [2]:
def generate_triplets(X, kin=50, kout = 10, kr = 5, weight_adj=False, random_triplets=True):
    num_extra = np.maximum(kin+50, 60) # look up more neighbors
    n = X.shape[0]
    nbrs = knn(n_neighbors= num_extra + 1, algorithm='auto').fit(X)
    distances, indices = nbrs.kneighbors(X)
#    sig = distances[:,10]
    sig = np.maximum(np.mean(distances[:, 10:20], axis=1), 1e-20) # scale parameter
    P = np.exp(-distances**2/np.reshape(sig[indices.flatten()],[n, num_extra + 1])/sig[:,np.newaxis])
    sort_indices = np.argsort(-P, axis = 1) # actual neighbors
    
    triplets = np.zeros([n * kin * kout, 3])
    weights = np.zeros(n * kin * kout)
    
    cnt = 0
    for i in range(n):
        for j in range(kin):
            sim = indices[i,sort_indices[i,j+1]]
            p_sim = P[i,sort_indices[i,j+1]]
            rem = indices[i,sort_indices[i,:j+2]].tolist()
            l = 0
            while (l < kout):
                out = np.random.choice(n)
                if out not in rem:
                    triplets[cnt,:] = [i, sim, out]
                    p_out = np.exp(-np.sum((X[i,:]-X[out,:])**2)/(sig[i] * sig[out]))
                    if p_out < 1e-20:
                        p_out = 1e-20
                    weights[cnt] = p_sim/p_out
                    rem.append(out)
                    l += 1
                    cnt += 1
        if ((i+1) % 10000) == 0:
            print ('Genareted triplets %d out of %d' % (i+1, n))
    if random_triplets:
        kr = 5
        triplets_rand = np.zeros([n * kr, 3])
        weights_rand = np.zeros(n * kr)
        for i in range(n):
            cnt = 0
            while cnt < kr:
                sim = np.random.choice(n)
                if sim == i:
                    continue
                out = np.random.choice(n)
                if out == i or out == sim:
                    continue
                p_sim = np.exp(-np.sum((X[i,:]-X[sim,:])**2)/(sig[i] * sig[sim]))
                if p_sim < 1e-20:
                    p_sim = 1e-20
                p_out = np.exp(-np.sum((X[i,:]-X[out,:])**2)/(sig[i] * sig[out]))
                if p_out < 1e-20:
                    p_out = 1e-20
                if p_sim < p_out:
                    sim, out = out, sim
                    p_sim, p_out = p_out, p_sim
                triplets_rand[i * kr + cnt,:] = [i, sim, out]
                weights_rand[i * kr + cnt] = p_sim/p_out
                cnt += 1
            if ((i+1) % 10000) == 0:
                print ('Genareted random triplets %d out of %d' % (i+1, n))
        triplets = np.vstack((triplets, triplets_rand))
        weights = np.hstack((weights, weights_rand))
    triplets = triplets[~np.isnan(weights),:]
    weights = weights[~np.isnan(weights)]
    weights /= np.max(weights)
    weights += 0.0001
    if weight_adj:
        weights = np.log(1 + 50 * weights)
        weights /= np.max(weights)
    return (triplets.astype(int), weights.flatten())

In [3]:
def trimap_grad(Y, triplets, weights):
    n, dim = Y.shape[0],Y.shape[1]
    #n = len(Y)
    #dim = 1
    grad = np.zeros([n, dim])
    y_ij = Y[triplets[:,0],:] - Y[triplets[:,1],:]
    y_ik = Y[triplets[:,0],:] - Y[triplets[:,2],:]
    d_ij = 1 + np.sum(y_ij**2,axis=-1)
    d_ik = 1 + np.sum(y_ik**2,axis=-1)
    num_viol = np.sum(d_ij > d_ik)
    denom = (d_ij + d_ik)**2
    loss = weights.dot(d_ij/(d_ij + d_ik))
    gs = 2 * y_ij * (d_ik/denom * weights)[:,np.newaxis]
    go = 2 * y_ik * (d_ij/denom * weights)[:,np.newaxis]
    for i in range(dim):
        grad[:,i] += np.bincount(triplets[:,0], weights= gs[:,i] - go[:,i])
        grad[:,i] += np.bincount(triplets[:,1], weights = -gs[:,i])
        grad[:,i] += np.bincount(triplets[:,2], weights = go[:,i])
    return (loss, grad, num_viol)

In [4]:
def trimap(X, num_dims = 2, num_neighbs = 50, num_out = 10, num_rand=5, eta = 1000.0, Yinit=[]):
    #print(X.shape)
    n, dim = X.shape[0], X.shape[1]
    #n = X.shape
    print("running TriMap on %d points with dimension %d" % (n,dim))
    print("PLEASE DO NOT DISTRIBUTE THE CODE!")
    X -= np.min(X)
    X = X/np.max(X)
    #np.divide(X,np.max(X), out=X )
    X = X - np.mean(X,axis=0)
    if dim > 50:
        X = TruncatedSVD(n_components=50, random_state=0).fit_transform(X)
    if np.size(Yinit) > 0:
        Y = Yinit
    else:
        Y = np.random.normal(size=[n, num_dims]) * 0.0001
    C = np.inf
    best_C = np.inf
    best_Y = Y
    tol = 1e-7
    num_iter = 2000
#    eta = 500.0 # learning rate
    
    triplets, weights = generate_triplets(X, num_neighbs, num_out, num_rand)
    num_triplets = float(triplets.shape[0])
    
    for itr in range(num_iter):
        old_C = C
        C, grad, num_viol = trimap_grad(Y, triplets, weights)
        
        # maintain the best answer
        if C < best_C:
            best_C = C
            best_Y = Y
            
        # update Y
        Y -= (eta/num_triplets * n) * grad;
        
        # update the learning rate
        if old_C > C + tol:
            eta = eta * 1.01
        else:
            eta = eta * 0.5
        
        if (itr+1) % 100 == 0:
            print ('Iteration: %4d, Loss: %3.3f, Violated triplets: %0.4f' % (itr+1, C, float(num_viol)/num_triplets))
    return best_Y

In [5]:
def main():
    data = pd.read_csv("fashion-mnist_train.csv")
    rndperm = data.sample(frac=1)
    X = np.array(data.drop(['label'],1))
    y = np.array(data['label'])
    X = X[:1000,]
    y = y[:1000,]
    #feat_cols = ['pixel' + str(i) for i in range(X.shape[1])]
    #df = pd.DataFrame(X, columns = feat_cols)
    #df['Label'] = y
    #df['Label'] = df['Label'].apply(lambda i:str(i))
    #print(df.shape)
    #rndperm = df.sample(frac=1)
    #X = df.drop(df.index[785],inplace=True)
    #y = df(df.index[785])
    #print(X.shape, y.shape)
    print(X.shape[0], X.shape[1], y.shape)
    
    Y = trimap(X, 2 , 50, 10, 5, 1000)
    joblib.dump(Y, 'trimapY.pkl')
    joblib.dump(y, 'trimaplabels.pkl')
    joblib.dump(X, 'trimapdata.pkl')
    #print(Y)
if __name__ == '__main__':
    main()

1000 784 (1000,)
running TriMap on 1000 points with dimension 784
PLEASE DO NOT DISTRIBUTE THE CODE!
Iteration:  100, Loss: 12.490, Violated triplets: 0.0955
Iteration:  200, Loss: 7.577, Violated triplets: 0.0525
Iteration:  300, Loss: 6.181, Violated triplets: 0.0502
Iteration:  400, Loss: 5.464, Violated triplets: 0.0487
Iteration:  500, Loss: 5.016, Violated triplets: 0.0470
Iteration:  600, Loss: 4.882, Violated triplets: 0.0463
Iteration:  700, Loss: 4.732, Violated triplets: 0.0455
Iteration:  800, Loss: 4.669, Violated triplets: 0.0451
Iteration:  900, Loss: 4.602, Violated triplets: 0.0446
Iteration: 1000, Loss: 4.556, Violated triplets: 0.0445
Iteration: 1100, Loss: 4.530, Violated triplets: 0.0445
Iteration: 1200, Loss: 4.503, Violated triplets: 0.0446
Iteration: 1300, Loss: 4.483, Violated triplets: 0.0446
Iteration: 1400, Loss: 4.469, Violated triplets: 0.0446
Iteration: 1500, Loss: 4.453, Violated triplets: 0.0446
Iteration: 1600, Loss: 4.441, Violated triplets: 0.0446
It